In [2]:
import json

In [3]:
database_path = '../data/cleaned_vlsp2025_law_db.json'
dataset_path = "../data/VLSP 2025 - MLQA-TSR Data Release/train_data/vlsp_2025_train.json"

with open(database_path, "r") as f:
    database = json.load(f)

with open(dataset_path, "r") as f:
    dataset = json.load(f)

In [19]:
print(type(database[0]))

<class 'dict'>


In [ ]:
import ast

new_database = []
for record in database:
    new_database.append(ast.literal_eval(record))

print(type(new_database))

In [16]:
for data in dataset:
    if data["question_type"] != "Yes/No":
        print(data)
        break

{'id': 'train_1', 'image_id': 'train_1_3', 'question': 'Biển báo cấm xe khách trên 29 chỗ được áp dụng trong các khoảng thời gian nào? ', 'relevant_articles': [{'law_id': 'QCVN 41:2024/BGTVT', 'article_id': '22'}], 'question_type': 'Multiple choice', 'choices': {'A': 'Từ 6:30 đến 8:00 và từ 16:30 đến 18:30; ngoài các khoảng thời gian này không được phép lưu thông.', 'B': 'Từ 6:30 đến 8:00 và từ 16:30 đến 18:30; ngoài các khoảng thời gian này được phép lưu thông.', 'C': 'Cấm lưu thông cả ngày.', 'D': 'D. Không cấm xe khách trên 29 chỗ lưu thông.'}, 'answer': 'B'}


# Construct the dataset json file

## Initialize the text embedding model

In [6]:
import torch
from sentence_transformers import SentenceTransformer

TEXT_MODEL_NAME = "BAAI/bge-m3"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(TEXT_MODEL_NAME, device=DEVICE)

In [7]:
from tqdm import tqdm

for record in tqdm(database, desc="Generating embeddings for the database"):
    pt_embedding = model.encode(record["text"], convert_to_tensor=True)
    np_embedding = pt_embedding.detach().cpu().numpy()
    list_embedding = np_embedding.tolist()
    record["embedding"] = list_embedding


Generating embeddings for the database:   0%|          | 0/399 [00:00<?, ?it/s]

Generating embeddings for the database: 100%|██████████| 399/399 [00:30<00:00, 13.15it/s]


In [ ]:
import random
dataset_image_base_path = "../data/VLSP 2025 - MLQA-TSR Data Release/train_data/images/"

new_dataset = []
max_pairs = 1
for data in dataset:
    image_file = data["image_id"] + ".jpg"
    query_text = ""
    if data["question_type"] == "Multiple choice":
        query_text = data["question"]+ "\n"
        for key, value in data["choices"].items():
            query_text += f"{key}: {value}\n"
    elif data["question_type"] == "Yes/No":
        query_text = data["question"] + "\n"
    relevant_articles = data["relevant_articles"]
    for article in relevant_articles:
        positive_article_id = article
        negative_article = {}
        
        # Get the embedding for the positive article
        positive_embedding = []
        for record in database:
            if record["law_id"] == positive_article_id["law_id"] and record["article_id"] == positive_article_id["article_id"]:
                positive_embedding = record["embedding"]
                positive_article = record["text"]
                break
        
        # Get random negative articles
        negative_articles = []
        for record in database:
            if record["law_id"] != positive_article_id["law_id"] or record["article_id"] != positive_article_id["article_id"]:
                negative_articles.append(record)
        negative_articles = random.sample(negative_articles, max_pairs)
        negative_embeddings = [record["embedding"] for record in negative_articles]
        negative_texts = [record["text"] for record in negative_articles]
        for negative_article in negative_articles:
            negative_text = negative_article["text"]
            negative_embedding = negative_article["embedding"]
            new_dataset.append({
                "image_file": image_file,
                "query_text": query_text,
                "positive_article": {
                    "text": positive_article,
                    "embedding": positive_embedding
                },
                "negative_articles": {
                    "text": negative_texts,
                    "embedding": negative_embeddings
                }
            })
        # negative_article = {
        #     "text": negative_texts,
        #     "embedding": negative_embeddings
        # }
        # new_dataset.append({
        #     "image_file": image_file,
        #     "query_text": query_text,
        #     "positive_article": {
        #         "text": positive_article,
        #         "embedding": positive_embedding
        #     },
        #     "negative_articles": negative_article
        # })




In [31]:
print(new_dataset[0]["negative_articles"])

{'text': ['a) Để báo trước sắp đến đoạn đường do sửa chữa hoặc có trở ngại ở một phía đường mà phải tổ chức đi lại cho phương tiện cả hai chiều trên phía đường còn lại hoặc để báo trước đoạn đường đôi tạm thời hoặc đoạn đường có chiều xe đi và về đi chung thì đặt biển số W.204 "Đường hai chiều".\nb) Các đoạn đầu và cuối đường có dải phân cách giữa chuyển tiếp sang đường đi chung hai chiều hoặc khi hết đoạn đường một chiều, đặt biển số W.204.\n\n\nHình C.4- Biển số W.204'], 'embedding': [[-0.038719236850738525, 0.023621320724487305, -0.024948185309767723, -0.004306533839553595, -0.04231776297092438, -0.024996278807520866, -0.006891448050737381, 0.04637601971626282, -0.012650293298065662, 0.037720467895269394, -0.04323849454522133, 0.03261173889040947, 0.041500382125377655, 0.005478628445416689, -0.021685071289539337, -0.008802521042525768, -0.005794992670416832, -0.012658899649977684, 0.005626281723380089, -0.02374325320124626, -0.013638557866215706, -0.00953966286033392, -0.02228971011

In [27]:
print(f"New dataset size: {len(new_dataset)}")

New dataset size: 1223


In [28]:
print(f"Dataset size: {len(dataset)}")

Dataset size: 530
